# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [13]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        '''ToDo: fix this function'''
        return 2.0 * sum(self.w) / (2.0 * self.C)

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        '''ToDo: fix this function'''
        return -1 if answer * self.f(x) < 1 else 0 

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train, k_power=0.95):
        self.w = np.random.rand(len(X_train[0])) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        step_const = 1 - 1e-4
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(1000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = step_const ** (k * k_power)

            # w update
            '''ToDo: add w update with regularization'''
            self.w -= step * y * x * (self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [14]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-10.60407509  -1.76480557   0.90345756  25.27970324 -13.36315214
  -2.89880888 -18.03667082   1.49339828  -8.24227879 -17.03136942
  -8.21537582   7.96375371  17.81141266   0.19633493  -6.02634006
   4.60996564   9.94625673  25.39355188   0.17478208   2.60629189] 8.83187629359704


In [15]:
predictions = model.predict(X_test)

In [16]:
print(predictions)

[0 1 1 ... 1 1 1]


In [17]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [18]:
print(len(predictions), sum(predictions))
mysvm_predict = sum(predictions == y_test) / float(len(y_test))

2000 1357


In [19]:
print(mysvm_predict)

0.678


# Найдем лучшие параметры для шага обучения

In [46]:
import operator
n_tests = 20
scores = {}
for i in range(1, n_tests):
    k_power = 1 - i/n_tests
    model.fit(X_train, y_train, k_power)
    predictions = model.predict(X_test)
    mysvm_predict = sum(predictions == y_test) / float(len(y_test))
    actual_score = {k_power:mysvm_predict}
    
    scores.update(actual_score)
    
best_k_power = max(scores.items(), key=operator.itemgetter(1))[0]
best_score = max(scores.values())

In [47]:
print(best_k_power, best_score)

0.85 0.723


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [48]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

linearsvc = LinearSVC()
linearsvc.fit(X_train, y_train)
linearsvc_predict = accuracy_score(linearsvc.predict(X_test), y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [49]:
print(linearsvc_predict)

0.7945


In [50]:
print("Точность моего SVM: {} vs Точность LinearSvm: {}".format(best_score, linearsvc_predict))

Точность моего SVM: 0.723 vs Точность LinearSvm: 0.7945
